# Van der Pol oscillator
We will look at the second order differentual equation (see https://en.wikipedia.org/wiki/Van_der_Pol_oscillator):

$$
{d^2y_0 \over dx^2}-\mu(1-y_0^2){dy_0 \over dx}+y_0= 0
$$

In [ ]:
from __future__ import division, print_function
import itertools
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from pyodesys.symbolic import SymbolicSys
sp.init_printing()
%matplotlib inline
print(sp.__version__)

One way to reduce the order of our second order differential equation is to formulate a system of first order ODEs, using:

$$ y_1 = \dot y_0 $$

which gives us:

$$
\begin{cases}
\dot y_0 = y_1 \\
\dot y_1 = \mu(1-y_0^2) y_1-y_0
\end{cases}
$$

Let's call this system of ordinary differential equations vdp1:

In [ ]:
vdp1 = lambda x, y, p: [y[1], -y[0] + p[0]*y[1]*(1 - y[0]**2)]

In [ ]:
y0 = [0, 1]
mu = 2.5
tend = 25

In [ ]:
odesys1 = SymbolicSys.from_callback(vdp1, 2, 1, names='y0 y1'.split())
odesys1.exprs

In [ ]:
# Let us plot using 30 data points
odesys1.integrate(np.linspace(0, tend, 30), y0, [mu], name='vode')
xout, yout = odesys1.plot_result()

If a smaller number of data points is desired, interpolation might be useful. Let us investigate some approaches:

In [ ]:
def solve_and_plot(sparse=0, roots=None, nderiv=0, interpolate=None, **kwargs):
    odesys2 = SymbolicSys(zip(odesys1.dep, odesys1.exprs), odesys1.indep, odesys1.params, roots=roots)
    xout2, yout2, info2 = odesys2.integrate([0, tend], y0, [mu], integrator='cvode', sparse=sparse, nderiv=nderiv)
    info2.pop('root_indices', None)
    if interpolate is None:
        interpolate = nderiv > 0
    plt.figure(figsize=(14, 8 if interpolate else 4))
    ax1 = plt.subplot(2 if interpolate else 1, 1, 1)
    c = ('k', 'r')
    xplot, yplot = odesys2.plot_result(interpolate=interpolate, m_lim=100,
                                       ls=('-',), c=c, **kwargs)
    plt.ylabel('y')
    
    if interpolate:
        xref, yref, inforef = odesys2.integrate(xplot, y0, [mu], integrator='cvode',
                                                force_predefined=True, nsteps=5000)
        for idx, y in enumerate(yref.T):
            plt.plot(xref, y, ls='--', c=c[idx], alpha=0.35)
        
        plt.subplots_adjust(hspace=0.0005)
        ax2 = plt.subplot(2, 1, 2, sharex=ax1)
        plt.ylabel('error')
        ydiff = yplot - yref
        for idx, y in enumerate(ydiff.T):
            plt.plot(xref, y, ls='-', c=c[idx])
        plt.setp(ax1.get_xticklabels(), visible=False)
    
    plt.xlabel('x')
    info2.pop('internal_xout')
    info2.pop('internal_yout')
    return info2, xplot, yplot

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1)  # this will just output data points at t0 and tend
info

Report for times where either derivative is zero:

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs, lines=0.3)
info

Use the first derivative in the interpolation

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs, nderiv=1)
info

Use first and second derivative in the interpolation

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs, nderiv=2)
info

Since the system is small we can formulate equations for where the second derivative is zero:

In [ ]:
d2fdx2 = tuple(odesys1.get_jac().dot(odesys1.exprs))
d2fdx2

Let's solve and report at all points where either y0'=0, y1'=0, y0''=0 or y1''=0

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs + d2fdx2)
info

Using the first derivative

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs + d2fdx2, nderiv=1)
info

And also the second

In [ ]:
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs + d2fdx2, nderiv=2)
info

In [ ]:
import scipy.interpolate
info, xplot, yplot = solve_and_plot(sparse=-1, roots=odesys1.exprs + d2fdx2, nderiv=0, interpolate=True,
                                    interp_from_deriv=scipy.interpolate.PchipInterpolator)